In [1]:
import webbrowser
import time
import json
import pandas as pd
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')
df = payload[4]
symb_list=df['Ticker'].values.tolist()

In [3]:
SEC_KEY = 'ZTqQ3wx4U4fH2rvAaN21M7rQLCq7O8qFn9ODRIXC'
PUB_KEY = 'PKGEW0G23998ZUZNR57H'
BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) # For real trading, don't enter a base_url


In [74]:
data=api.get_bars(symb_list, TimeFrame.Day, "2022-07-25",  "2022-07-29", limit=9999, adjustment='raw')
pd_data=data.df

In [51]:
def get_data(symb_list,frequency,start,end,limit):
    data=api.get_bars(symb_list, frequency, start,  end, limit=9999, adjustment='raw')
    pd_data=data.df
    subset=pd_data[['symbol','close']]
    return subset

def sort_data(data):
    new=data.groupby("symbol")
    nod=len(new.get_group(data["symbol"][0]))#num of days
    price_list=[]
    symbol_list=[]
    change_list=[]
    for i in range(int(len(data)/nod)):
        dif=data['close'][nod*(i+1)-1]-data['close'][i*nod]
        avg=sum(data['close'][nod*i:nod*(i+1)])/nod
        symbol_list.append(data['symbol'][nod*i])
        price_list.append(data['close'][nod*(i+1)-1])
        change_list.append(dif/avg)
    data=pd.DataFrame(list(zip(symbol_list,price_list,change_list)),columns=['symb','price','change'])
    sorted_data=data.sort_values(by=['change'])
    sorted_data = sorted_data.reset_index(drop=True)
    return sorted_data
def place_orders(data,N,notional):
    for index, row in data.iterrows():
        if index<N:
            api.submit_order(
          symbol=row["symb"], # Replace with the ticker of the stock you want to buy
          #notional=str(notional),
          qty=int(notional/row["price"]),
          side='buy',
          type='limit', 
          limit_price=str(row["price"]),
          time_in_force='day'
            )
def place_orders2(data):
    for index, row in data.iterrows():
        api.submit_order(
        symbol=row["symb"], # Replace with the ticker of the stock you want to buy
        #notional=str(notional),
        qty=str(int(row["qty"])-int(row["filled_qty"])),
        side='buy',
        type='limit', 
        #limit_price=str(float(row["limit_price"])*1.001),
        limit_price=str(row["price"]),
        time_in_force='day'
        )
def get_latest_price(symb_list):
    latest_list={}
    latest_data=api.get_latest_bars(symb_list)
    for symb in symb_list:
        datapoint=latest_data[symb]._raw
        price=datapoint['l']
        latest_list[symb]=price
    return latest_list
        
    
def get_orders_status():
    orders=api.list_orders()
    avg=[]
    symbol=[]
    qty=[]
    filled_qty=[]
    limit_price=[]
    for order in orders:
        order=order._raw
        avg.append(order['filled_avg_price'])
        symbol.append(order['symbol'])
        qty.append(order['qty'])
        filled_qty.append(order['filled_qty'])
        limit_price.append(order['limit_price'])
    latest_list=get_latest_price(symbol)
    order_data=pd.DataFrame(list(zip(symbol,qty,filled_qty,avg,limit_price)),columns=['symb','qty','filled_qty','avg','price'])
    #order_data=pd.DataFrame(list(zip(symbol,qty,filled_qty,avg,latest_list)),columns=['symb','qty','filled_qty','avg','price'])
    #latest_price=api.get_latest_bars(symbol)   
    return order_data 



def cancel_all_orders():
    api.cancel_all_orders()       

In [43]:

# place_orders(sorted_data,5,5000)
# orders=api.list_orders()
# orders[2]

In [52]:
data=get_data(symb_list,TimeFrame.Day, "2022-07-25",  "2022-07-29", limit=9999)
#print(data)
sorted_data=sort_data(data)
print(sorted_data)

      symb   price    change
0      PDD   49.01 -0.135196
1    CMCSA   37.52 -0.120444
2     CHTR  432.10 -0.101842
3     INTC   36.31 -0.073333
4       JD   59.50 -0.055967
..     ...     ...       ...
97    CDNS  186.08  0.104100
98    ALGN  280.97  0.108666
99    AMZN  134.95  0.112432
100   DDOG  102.01  0.123077
101    CEG   66.10  0.187744

[102 rows x 3 columns]


In [53]:
api.cancel_all_orders()
place_orders(sorted_data,5,5000)
rest=get_orders_status()
for i in range(2):
    place_orders2(rest)
    time.sleep(10)
    rest=get_orders_status()
    cancel_all_orders()
    time.sleep(10)


ValueError: could not convert string to float: 'JD'

In [14]:
#print(orders_history[0][0]._raw)
avg=[]
symbol=[]
qty=[]
filled_qty=[]
for order in orders_history[0]:
    order=order._raw
    avg.append(order['filled_avg_price'])
    symbol.append(order['symbol'])
    qty.append(order['qty'])
    filled_qty.append(order['filled_qty'])
order_data=pd.DataFrame(list(zip(symbol,qty,filled_qty,avg)),columns=['symb','qty','filled_qty','avg'])
order_data.head()   

,symb,qty,filled_qty,avg
0,JD,84,0,None
1,CHTR,11,0,None
2,CMCSA,133,0,None
3,PDD,102,0,None


In [38]:
class auto_trade(self,):
    def get_api(SEC_KEY,PUB_KEY,BASE_URL):
        self.api=tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)
    
    def get_data(start,end,frequency,type):
        data=self.api.get_bars(symb_list, TimeFrame.Day, "2022-07-25",  "2022-07-29", limit=9999, adjustment='raw')
        if type=="raw":
            self.data=data
        if type=="pd":
            self.data=


In [45]:
new=subset.groupby("symbol")
len(new.get_group(subset["symbol"][0]))
#new.head()

5

In [39]:
my_watch_list=api.create_watchlist("my_list")
my_watch_list

Watchlist({   'account_id': '121ac014-6975-4bc4-8888-1ed671fb66e6',
    'assets': [],
    'created_at': '2022-08-02T19:26:05.538292Z',
    'id': '0c44305a-df24-487c-93e8-f37cf538ad01',
    'name': 'my_list',
    'updated_at': '2022-08-02T19:26:05.538292Z'})

In [40]:
?api.get_latest_bars

In [54]:
latest_data=api.get_latest_bars(symb_list)
latest_data

{'GOOGL': BarV2({   'c': 115.32,
     'h': 115.37,
     'l': 115.32,
     'n': 49,
     'o': 115.35,
     't': '2022-08-02T19:43:00Z',
     'v': 1849,
     'vw': 115.351952}),
 'FTNT': BarV2({   'c': 60.6,
     'h': 60.62,
     'l': 60.56,
     'n': 26,
     'o': 60.57,
     't': '2022-08-02T19:43:00Z',
     'v': 1292,
     'vw': 60.587252}),
 'VRSN': BarV2({   'c': 193.68,
     'h': 193.68,
     'l': 193.64,
     'n': 3,
     'o': 193.64,
     't': '2022-08-02T19:42:00Z',
     'v': 233,
     'vw': 193.661459}),
 'COST': BarV2({   'c': 544.83,
     'h': 544.83,
     'l': 544.765,
     'n': 12,
     'o': 544.765,
     't': '2022-08-02T19:43:00Z',
     'v': 345,
     'vw': 544.778043}),
 'BIDU': BarV2({   'c': 134.65,
     'h': 134.65,
     'l': 134.65,
     'n': 4,
     'o': 134.65,
     't': '2022-08-02T19:43:00Z',
     'v': 246,
     'vw': 134.723659}),
 'AZN': BarV2({   'c': 65.445,
     'h': 65.445,
     'l': 65.43,
     'n': 18,
     'o': 65.43,
     't': '2022-08-02T19:43:00Z',
  

In [69]:
# account=api.get_account()
# accou

In [72]:
PDD=api.get_trades('PDD')
PDD

[Trade({   'c': ['@', 'T', 'I'],
     'i': 1,
     'p': 46.35,
     's': 1,
     't': '2022-08-02T08:00:00.044115456Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'F', 'T', 'I'],
     'i': 2,
     'p': 46.35,
     's': 1,
     't': '2022-08-02T08:00:02.124014848Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'T', 'I'],
     'i': 3,
     'p': 46,
     's': 1,
     't': '2022-08-02T08:00:09.359012352Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'T', 'I'],
     'i': 4,
     'p': 46,
     's': 6,
     't': '2022-08-02T08:00:09.359012352Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'T', 'I'],
     'i': 5,
     'p': 46,
     's': 3,
     't': '2022-08-02T08:00:09.359012352Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'T', 'I'],
     'i': 6,
     'p': 45.89,
     's': 90,
     't': '2022-08-02T08:00:09.359012352Z',
     'x': 'P',
     'z': 'C'}),
 Trade({   'c': ['@', 'F', 'T'],
     'i': 7,
     'p': 46.35,
     's': 319,
     't': '20

In [74]:
?api.entity_v2.TradesV2

Object `api.entity_v2.TradesV2` not found.


In [63]:
api.get_position('PDD')

Position({   'asset_class': 'us_equity',
    'asset_id': 'bce94a21-f7e5-4e99-a2f4-9d76514742c4',
    'asset_marginable': True,
    'avg_entry_price': '48.99875',
    'change_today': '0.029356259173831',
    'cost_basis': '39982.98',
    'current_price': '49.09',
    'exchange': 'NASDAQ',
    'lastday_price': '47.69',
    'market_value': '40057.44',
    'qty': '816',
    'qty_available': '816',
    'side': 'long',
    'symbol': 'PDD',
    'unrealized_intraday_pl': '74.46',
    'unrealized_intraday_plpc': '0.0018622924054185',
    'unrealized_pl': '74.46',
    'unrealized_plpc': '0.0018622924054185'})

In [68]:
position_data=api.list_positions()
position_data[0]._raw

{'asset_id': '0cf42aa3-9816-4f1f-aa84-6482ac9303e9',
 'symbol': 'INTC',
 'exchange': 'NASDAQ',
 'asset_class': 'us_equity',
 'asset_marginable': True,
 'qty': '2055',
 'avg_entry_price': '36.1846666666666667',
 'side': 'long',
 'market_value': '74185.5',
 'cost_basis': '74359.49',
 'unrealized_pl': '-173.99',
 'unrealized_plpc': '-0.0023398492915968',
 'unrealized_intraday_pl': '-173.99',
 'unrealized_intraday_plpc': '-0.0023398492915968',
 'current_price': '36.1',
 'lastday_price': '36.96',
 'change_today': '-0.0232683982683983',
 'qty_available': '2055'}